In [ ]:
# Load packages
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
import time
from datetime import datetime


# Load my own functions
import dnnregressor_train_eval_keras as dnnk
from kde_training_utilities import kde_load_data
from kde_training_utilities import kde_make_train_test_split
import make_data_wfpt as mdw

In [ ]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# Make dnnk class (cpm for choice probability model)
cpm = dnnk.dnn_trainer()

# Define folder in which dataset lies
data_folder = '/media/data_cifs/afengler/data/kde/ddm/train_test_data/'

In [ ]:
# Make train test split
kde_make_train_test_split(folder = data_folder,
                          p_train = 0.8)

In [ ]:
# Load train test split
cpm.data['train_features'], cpm.data['train_labels'], cpm.data['test_features'], cpm.data['test_labels'] = kde_load_data(folder = data_folder)

In [ ]:
cpm.data['train_features'].shape

In [ ]:
cpm.data['train_features']

In [ ]:
cpm.data['train_features'].iloc[171247010, :]

In [ ]:
cpm.data['train_features']['log_l'] = cpm.data['train_labels']

In [ ]:
cpm.data['train_features'].sort_values(by = 'log_l')

In [ ]:
cpm.data['train_features']

In [ ]:
cpm.data['train_features'].iloc[22428, :]

In [ ]:
cpm.data['train_labels'][22428, ]

In [ ]:
# Make all parameters we can specify explicit
# Model parameters
cpm.model_params

In [ ]:
# Parameters governing training
cpm.train_params

In [ ]:
# Parameters concerning data storage
cpm.data_params

In [ ]:
# If necessary, specify new set of parameters here:
# Model params
cpm.model_params['output_activation'] = 'linear'
cpm.model_params['hidden_layers'] = [20, 40, 60, 80, 100, 120]
cpm.model_params['hidden_activations'] = ['relu', 'relu', 'relu', 'relu', 'relu', 'relu']
cpm.model_params['input_shape'] = cpm.data['train_features'].shape[1]
# cpm.model_params['l1_activation'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# cpm.model_params['l2_activation'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cpm.model_params['l1_kernel'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cpm.model_params['l2_kernel'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Train params
cpm.train_params['batch_size'] = 1000000
cpm.train_params['max_train_epochs'] = 250
cpm.train_params['min_delta'] = 0.00001


# Data params
cpm.data_params['data_type'] = 'kde'
cpm.data_params['data_type_signature'] = '_ddm_'
cpm.data_params['training_data_size'] = cpm.data['train_features'].shape[0]
cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
cpm.data_params['model_directory'] = '/media/data_cifs/afengler/data/kde/ddm/keras_models/'

In [ ]:
# Make model
cpm.keras_model_generate(save_model = True)

In [ ]:
# Train model
cpm.run_training(save_history = True, 
                 warm_start = False)